In [6]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [15]:
def create_data_model():
    data = {}
    data['distance_matrix'] = [[0.        ,  9.08150584, 11.52262012,  5.8282368 , 17.64029965],
       [ 9.08150584,  0.        , 19.56945236,  6.977526  , 19.91500572],
       [11.52262012, 19.56945236,  0.        , 13.18741504, 14.06403735],
       [ 5.8282368 ,  6.977526  , 13.18741504,  0.        , 13.72528908],
       [17.64029965, 19.91500572, 14.06403735, 13.72528908,  0.        ]]
    
    data['num_vehicles'] = 2
    data['depot'] = 0
    data['demands'] = [0, 3, 5, 2, 6]
    data['vehicle_capacities'] = [10, 8]
    
    

    
    
    
    
    return data

In [16]:
def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'] )
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    
#     def time_callback(from_index, to_index):
#         from_node = manager.IndexToNode(from_index)
#         to_node = manager.IndexToNode(to_node)
#         return data['time_windows'][from_node][to_node]
    
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    
    
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


        
    
    
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    
    search_parameters.time_limit.FromSeconds(1)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("no solution")

In [17]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


In [18]:
main()

Objective: 73
Route for vehicle 0:
 0 Load(0) ->  1 Load(3) ->  3 Load(5) ->  2 Load(10) ->  0 Load(10)
Distance of the route: 39m
Load of the route: 10

Route for vehicle 1:
 0 Load(0) ->  4 Load(6) ->  0 Load(6)
Distance of the route: 34m
Load of the route: 6

Total distance of all routes: 73m
Total load of all routes: 16
